In [9]:
from twitter_reader.twitter_manager.component import APIConfig, APIManager
from twitter_reader.data_storer.component import DBConfig, DBManager
from twitter_reader.tweets_transformer.component import text_only

In [2]:
tm_conf = APIConfig()
tm = APIManager(tm_conf)
tweets = tm.query('Superleague lang:en', pages=1)

https://api.twitter.com/2/tweets/search/recent?query=Superleague%20lang%3Aen&max_results=10&expansions=referenced_tweets.id&tweet.fields=text
--
got 10 tweets.


In [13]:
text_list = list(map(text_only, [t.text for t in tweets]))

In [14]:
text_list

['SuperLeague has been officially SUSPENDED - English clubs have left (Chelsea too, official soon). - Inter have lef…',
 'Real Madrid, Barcelona &amp; Juventus in the SuperLeague. https://t.co/FlZcVVBviN',
 'SUPERLEAGUE How Its Started-----------------------How It is Going    https://t.co/guCnKkePP7',
 'Official statement. Inter have left the SuperLeague, confirmed. “Inter confirms that the club is no longer a membe…',
 'Real Madrid, Barcelona &amp; Juventus in the SuperLeague. https://t.co/FlZcVVBviN',
 "Funny, I didn't spot any police crackdowns on unmasked mass gatherings when all those football fans celebrated the end…",
 'Football fans when they heard Chelsea, Man City, Barcelona &amp; Atletico are withdrawing from SuperLeague https://t.co/g…',
 "For those who follow soccer, remember when WWE SuperLeague'd the British independent wrestling scene and ruined it?",
 'The 12 clubs who committed to the ESL combine a total of 1.3 billion followers across the major social media channels 

In [10]:
doc_text = ". ".join([x['text'] for x in tweets])

In [11]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(doc_text)
for token in doc:
    print(token.text)

Nike
Air
Max
1
'
Dark
Teal
Green
'


Price:$150


Release
Date
:
2021
-
04
-
15


Style
Code
:
DH4059
-
101


Color
:
White
/
Dark
Teal
Green
/
Wolf
Grey
/
Black


=
&
gt;https://t.co
/
nH35N4KClz
https://t.co/ssHsqPDsHt
.
Nike
Air
Max
1
'
Dark
Teal
Green
'


Price:$150


Release
Date
:
2021
-
04
-
15


Style
Code
:
DH4059
-
101


Color
:
White
/
Dark
Teal
Green
/
Wolf
Grey
/
Black


=
&
gt;https://t.co
/
BnEpP2H8Vb
https://t.co/pC50DWYQAf
.
RT
@lfvm26
:
Que
lindo
es
el
amor
pero
son
más
lindas
unas
Nike
nuevas
🥰
.
Gente
como
a
gente
que
mistura
adidas
com
Nike
hahahaha
https://t.co/zGtXmwWnQ6
.
Nike
Air
Structure
"
Neo
Teal
"
is
available
under
retail
+
free
shipping



Buy
Here
:
https://t.co/qG0SOA1A0
T
https://t.co/Wtxx491alv
.
Few
sizes
restocked


Nike
Air
Structure
OG
'
Neo
Teal
'


=
&
gt
;
https://t.co/y3MfWFeA9O



15
%
off
w/
code
PARAPR15
https://t.co/QK5lryeLoL.
RT
@trajesRj
:
EGITO
X
NIKE
👏
🏿
https://t.co/ukLZ2zFajL.
Something
about
Z
wearing
a
nike
beanie
and
an
adidas
h

In [1]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

from spacy.language import Language

In [2]:
nytimes = pd.read_csv('./datasets/nytimes.csv')
newest_doc = nytimes['lead_paragraph']

In [3]:
newest_doc.head()

0    At Hostos Community College in the South Bronx...
1    WASHINGTON -- Companies spend millions of doll...
2    President Reagan today raised the question of ...
3    When flight attendants were ordered by their u...
4    THE escalation of violence in the Middle East ...
Name: lead_paragraph, dtype: object

In [4]:
nlp = spacy.load("en_core_web_sm")

# My list of stop words.
stop_list = ["Mrs.","Ms.","say","WASHINGTON","'s","Mr.",]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [5]:
@Language.component('lemmer')
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
@Language.component('stopper')    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
# nlp.add_pipe('lemmer',name='lemmatizer',after='ner')
nlp.add_pipe('stopper', name="stopwords", last=True)

<function __main__.remove_stopwords(doc)>

In [6]:
doc_list = []
# Iterates through each article in the corpus.
for doc in tqdm(newest_doc):
    # Passes that article through the pipeline and adds to a new list.b
    pr = nlp(doc)
    doc_list.append(pr)

<ipython-input-6-fb7e028ad7c5>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(newest_doc):


  0%|          | 0/5000 [00:00<?, ?it/s]

In [8]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=2, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
pprint(lda_model.print_topics(num_words=2))

[(0, '0.019*"percent" + 0.010*"said"'),
 (1, '0.008*"President" + 0.006*"economic"')]
